In [131]:
import pandas as pd
import numpy as np
import os
data_dir = "../data"+os.path.sep

#### i) Change in 3 month yield

In [114]:
DTB3 = pd.read_csv(data_dir+"DTB3.csv")
DTB3 = DTB3[DTB3.DTB3 != "."]
DTB3.DTB3 = DTB3.DTB3.astype(float)/100
DTB3.index = pd.to_datetime(DTB3.DATE, format="%Y-%m-%d")
DTB3 = DTB3.drop(["DATE"], axis=1).diff().dropna()
DTB3

,DTB3
DATE,
1954-01-05,-0.0005
1954-01-06,0.0000
1954-01-07,0.0003
1954-01-08,0.0000
1954-01-11,0.0002
...,...
2020-09-25,0.0000
2020-09-28,0.0001
2020-09-29,-0.0002


#### ii)  Change in the slope of the yield curve

In [81]:
# Data comes from two different series: USTREASURY-LONGTERMRATES for >Year 2000 and DLTBOARD for 1981-2000
LONGRATES = pd.read_csv(data_dir+"USTREASURY-LONGTERMRATES.csv")
DLTBOARD = pd.read_csv(data_dir+"DLTBOARD.csv")
DLTBOARD.index = pd.to_datetime(DLTBOARD.DATE, format="%Y-%m-%d")
DLTBOARD = DLTBOARD.drop(["DATE"], axis=1)
LONGRATES.index = pd.to_datetime(LONGRATES.Date, format="%Y-%m-%d")
LONGRATES = LONGRATES.drop(["Date", "Treasury 20-Yr CMT", "Extrapolation Factor"], axis=1)
LONGRATES = LONGRATES.rename({"LT Composite > 10 Yrs": "DLTBOARD"}, axis=1)
DLTBOARD = pd.concat([LONGRATES, DLTBOARD]).sort_index()
DLTBOARD = DLTBOARD[DLTBOARD.DLTBOARD != "."]
DLTBOARD.DLTBOARD = DLTBOARD.DLTBOARD.astype(float)/100

CHANGESLOPE = pd.DataFrame(DLTBOARD.DLTBOARD - DTB3.DTB3, columns=["CHANGESLOPE"]).dropna()
CHANGESLOPE

,CHANGESLOPE
1981-07-01,0.1255
1981-07-02,0.1299
1981-07-06,0.1281
1981-07-07,0.1237
1981-07-08,0.1285
...,...
2020-09-24,0.0123
2020-09-25,0.0122
2020-09-28,0.0122
2020-09-29,0.0125


#### iii) TED spread

In [118]:
TEDRATE = pd.read_csv(data_dir+"TEDRATE.csv")
TEDRATE = TEDRATE[TEDRATE.TEDRATE != "."]
TEDRATE.TEDRATE = TEDRATE.TEDRATE.astype(float).dropna()/100
TEDRATE.index = pd.to_datetime(TEDRATE.DATE, format="%Y-%m-%d")
TEDRATE = TEDRATE.drop(["DATE"], axis=1)
TEDRATE

,TEDRATE
DATE,
1986-01-02,0.0090
1986-01-03,0.0099
1986-01-06,0.0107
1986-01-07,0.0109
1986-01-08,0.0083
...,...
2020-09-23,0.0012
2020-09-24,0.0013
2020-09-25,0.0012


#### iv) Change in credit spread

In [113]:
DGS10 = pd.read_csv(data_dir+"DGS10.csv")
DGS10 = DGS10[DGS10.DGS10 != "."]
DGS10.DGS10 = DGS10.DGS10.astype(float).dropna()/100
DGS10.index = pd.to_datetime(DGS10.DATE, format="%Y-%m-%d")
DGS10 = DGS10.drop(["DATE"], axis=1)

DBAA = pd.read_csv(data_dir+"DBAA.csv")
DBAA = DBAA[DBAA.DBAA != "."]
DBAA.DBAA = DBAA.DBAA.astype(float).dropna()/100
DBAA.index = pd.to_datetime(DBAA.DATE, format="%Y-%m-%d")
DBAA = DBAA.drop(["DATE"], axis=1)

CHNGCREDITSPREAD = pd.DataFrame(DBAA.DBAA - DGS10.DGS10, columns=["CHNGCREDITSPREAD"]).dropna()
CHNGCREDITSPREAD

,CHNGCREDITSPREAD
DATE,
1986-01-02,0.0234
1986-01-03,0.0230
1986-01-06,0.0229
1986-01-07,0.0235
1986-01-08,0.0221
...,...
2020-09-25,0.0274
2020-09-28,0.0276
2020-09-29,0.0274


#### v) Market Return

In [132]:
MKT = pd.read_csv(data_dir+"CRSP_SP500 Return.csv")
MKT.index = pd.to_datetime(MKT.caldt, format="%Y%m%d")
MKT = MKT[["sprtrn"]].dropna()
DailyReturn = MKT.rename({"sprtrn": "DailyReturn"}, axis=1)
DailyReturn

,DailyReturn
caldt,
1962-07-03,0.011278
1962-07-05,0.005665
1962-07-06,-0.011266
1962-07-09,0.006765
1962-07-10,0.011494
...,...
2019-12-24,-0.000195
2019-12-26,0.005128
2019-12-27,0.000034


#### vi) Equity Volatility

In [136]:
ReturnVol  = DailyReturn.rolling(22).std()
ReturnVol = ReturnVol.rename({"DailyReturn": "EquityVol"}, axis=1).dropna()
ReturnVol

,EquityVol
caldt,
1962-08-02,0.008235
1962-08-03,0.007957
1962-08-06,0.008058
1962-08-07,0.007789
1962-08-08,0.007692
...,...
2019-12-24,0.004717
2019-12-26,0.004772
2019-12-27,0.004615
